In [7]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [8]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

In [9]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

In [10]:
model = WideAndDeepModel(30, activation='relu', name='dynamic_model')

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_data=((X_valid_wide, X_valid_deep), y_valid))

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 4.0813 - output_1_loss: 2.3386 - output_2_loss: 1.7426 - output_1_root_mean_squared_error: 1.5293 - output_2_root_mean_squared_error: 1.3201 - val_loss: 1.9449 - val_output_1_loss: 1.1198 - val_output_2_loss: 0.8251 - val_output_1_root_mean_squared_error: 1.0582 - val_output_2_root_mean_squared_error: 0.9083
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 1.3608 - output_1_loss: 0.6971 - output_2_loss: 0.6637 - output_1_root_mean_squared_error: 0.8349 - output_2_root_mean_squared_error: 0.8147 - val_loss: 7.9693 - val_output_1_loss: 4.0413 - val_output_2_loss: 3.9280 - val_output_1_root_mean_squared_error: 2.0103 - val_output_2_root_mean_squared_error: 1.9819
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 1.1697 - output_1_loss: 0.5694 - output_2_loss: 0.6003 - output_1_root_mean_squared_error: 0.7546 - output_2_root_mean_squared_error: 0.7748 - val_loss: 2.225

In [12]:
eval_results_dict = model.evaluate((X_test_wide, X_test_deep), y_test, return_dict=True)
eval_results_dict

162/162 [==============================] - 0s 1ms/step - loss: 0.7182 - output_1_loss: 0.3310 - output_2_loss: 0.3872 - output_1_root_mean_squared_error: 0.5753 - output_2_root_mean_squared_error: 0.6222


{'loss': 0.7181921601295471,
 'output_1_loss': 0.33102044463157654,
 'output_2_loss': 0.3871718943119049,
 'output_1_root_mean_squared_error': 0.5753437876701355,
 'output_2_root_mean_squared_error': 0.6222313642501831}

Save/load model

In [13]:
model.save('test_models/dynamic_model', save_format='tf')

INFO:tensorflow:Assets written to: test_models/dynamic_model\assets


INFO:tensorflow:Assets written to: test_models/dynamic_model\assets


In [14]:
model = tf.keras.models.load_model('test_models/dynamic_model')
eval_results_dict = model.evaluate((X_test_wide, X_test_deep), y_test, return_dict=True)
eval_results_dict

162/162 [==============================] - 1s 2ms/step - loss: 0.7182 - output_1_loss: 0.3310 - output_2_loss: 0.3872 - output_1_root_mean_squared_error: 0.5753 - output_2_root_mean_squared_error: 0.6222


{'loss': 0.7181921601295471,
 'output_1_loss': 0.33102044463157654,
 'output_2_loss': 0.3871718943119049,
 'output_1_root_mean_squared_error': 0.5753437876701355,
 'output_2_root_mean_squared_error': 0.6222313642501831}

Callbacks - Checkpoints and early stopping

In [15]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        ratio = logs['val_loss'] / logs['loss']
        print(f"Epoch={epoch}, val./tr.={ratio:.2f}")

In [16]:
model_b = WideAndDeepModel(30, activation='relu', name='dynamic_model_b')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model_b.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
model_b.norm_layer_wide.adapt(X_train_wide)
model_b.norm_layer_deep.adapt(X_train_deep)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('test_checkpoints/model_b', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
custom_cb = PrintValTrainRatioCallback()
history = model_b.fit((X_train_wide, X_train_deep), y_train, epochs=50, validation_data=((X_valid_wide, X_valid_deep), y_valid), 
                      callbacks=[checkpoint_cb, early_stopping_cb, custom_cb])

Epoch 1/50
363/363 [==============================] - 2s 2ms/step - loss: 3.3183 - output_1_loss: 1.5732 - output_2_loss: 1.7451 - output_1_root_mean_squared_error: 1.2543 - output_2_root_mean_squared_error: 1.3210 - val_loss: 5.8110 - val_output_1_loss: 3.4114 - val_output_2_loss: 2.3996 - val_output_1_root_mean_squared_error: 1.8470 - val_output_2_root_mean_squared_error: 1.5491
Epoch 2/50
363/363 [==============================] - 1s 2ms/step - loss: 1.1519 - output_1_loss: 0.5217 - output_2_loss: 0.6302 - output_1_root_mean_squared_error: 0.7223 - output_2_root_mean_squared_error: 0.7938 - val_loss: 1.0628 - val_output_1_loss: 0.4409 - val_output_2_loss: 0.6220 - val_output_1_root_mean_squared_error: 0.6640 - val_output_2_root_mean_squared_error: 0.7886
Epoch 3/50
363/363 [==============================] - 1s 2ms/step - loss: 0.9989 - output_1_loss: 0.4399 - output_2_loss: 0.5589 - output_1_root_mean_squared_error: 0.6633 - output_2_root_mean_squared_error: 0.7476 - val_loss: 1.091